## 1. Get pretrained model

1. download the facial emotion recognition https://github.com/HSE-asavchenko/face-emotion-recognition.git


2. Put this file into "src" folder

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd
import pickle
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize

%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
# from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
# from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
# from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.compat.v1.keras.backend import set_session 

INPUT_SIZE = (224, 224)

2023-10-30 12:48:19.564956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nali/.local/lib/python3.8/site-packages/cv2/../../lib64::/usr/lib64
2023-10-30 12:48:19.564973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from facial_analysis import FacialImageProcessing

In [5]:
imgProcessing=FacialImageProcessing(False) 

2023-10-30 12:48:23.201701: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-30 12:48:23.201726: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nali-Alienware-m15-R2): /proc/driver/nvidia/version does not exist
2023-10-30 12:48:23.202055: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 2. Emotion Analysis

In [6]:
model=load_model('../models/affectnet_emotions/mobilenet_7.h5')
model.summary()
USE_PREPROCESSING=True
INPUT_SIZE = (224, 224)
print(INPUT_SIZE)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                           

In [7]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
emotion_threeCategroies = {0: 'Negative', 1: 'Positive', 2: 'Neutral'}
emotion_fourCategories = {0: 'Negative', 1: 'Positive', 2: 'Neutral', 3: 'Surprise'}

In [10]:
# imgProcessing=FacialImageProcessing(False)

In [8]:
emotion_data = {'Anger': [0], 
        'Disgust': [0],
        'Fear':[0],
        'Happiness':[0],
        'Neutral':[0],
        'Sadness':[0],
        'Surprise':[0]
       }

In [9]:
emotion_categories ={'Negative':[0],
                    'Positive':[0],
                    'Neutral':[0],
                    'Surprise':[0]}

In [10]:
emotion_df = pd.DataFrame(emotion_data)

In [11]:
emotion_df_B = pd.DataFrame(emotion_data)  

In [12]:
emotion_df_A = pd.DataFrame(emotion_data)

In [13]:
emotion_df_A1_beforehelp = pd.DataFrame(emotion_data)
emotion_df_A1_beforehelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [14]:
emotion_df_A1_withhelp = pd.DataFrame(emotion_data)
emotion_df_A1_withhelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [15]:
emotion_df_B_A1 = pd.DataFrame(emotion_data)
emotion_df_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [16]:
emotion_df_A2 = pd.DataFrame(emotion_data)
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [17]:
emotion_df_B_A2 = pd.DataFrame(emotion_data)
emotion_df_B_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [18]:
emotion_df.iloc[[0],[0] ]= 0
emotion_df

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [19]:
emotion_df_categories = pd.DataFrame(emotion_categories)
emotion_df_categories.iloc[[0],[1]] = 0
emotion_df_categories

,Negative,Positive,Neutral,Surprise
0,0,0,0,0


In [20]:
df_emotion_frame = pd.DataFrame(columns=['image_ID', 'score', 'emotion', 'category'])
df_emotion_frame

,image_ID,score,emotion,category


In [21]:
df_emotion_frame_B_A1 = pd.DataFrame(columns=['image_ID', 'score', 'emotion', 'category'])
df_emotion_frame_B_A1

,image_ID,score,emotion,category


In [22]:
df_emotion_frame_B_A2 = pd.DataFrame(columns=['image_ID', 'score', 'emotion', 'category'])
df_emotion_frame_B_A2

,image_ID,score,emotion,category


In [23]:
df_emotion_frame_A2 = pd.DataFrame(columns=['image_ID', 'score', 'emotion', 'category'])
df_emotion_frame_A2

,image_ID,score,emotion,category


In [24]:
emotion_df

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [25]:
emotion_df_A1_withhelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [26]:
emotion_df_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [27]:
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [28]:
emotion_df_B_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [29]:
# idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
emotion_fourCategories = {0: 'Negative', 1: 'Positive', 2: 'Neutral', 3: 'Surprise'}

In [30]:
df_emotion_frame

,image_ID,score,emotion,category


In [149]:
def predictionEm(files,folderpath):
    index = 0
    index_failed = 0
    
    for fname in files:
        
        if index%1000 == 0:
            print("amount images:", index)
            print("faild images:", index_failed)
        
        fpath = folderpath + fname
        frame_bgr=cv2.imread(fpath)
        frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        bounding_boxes, points = imgProcessing.detect_faces(frame)
        points = points.T
    
        for bbox,p in zip(bounding_boxes, points):
            box = bbox.astype(np.int32)
            x1,y1,x2,y2=box[0:4]    
            face_img=frame[y1:y2,x1:x2,:]

            try:
                user_id = fname.split('_')[0]
                
                face_img=cv2.resize(face_img,INPUT_SIZE)
                
                inp=face_img.astype(np.float32)
                inp[..., 0] -= 103.939
                inp[..., 1] -= 116.779
                inp[..., 2] -= 123.68
                inp = np.expand_dims(inp, axis=0)

                scores=model.predict(inp)[0]
                if np.argmax(scores) in (0, 1, 2, 5):
                    categoryscore = 0
                elif np.argmax(scores) == 3:
                    categoryscore = 1
                elif np.argmax(scores) == 6:
                    categoryscore = 3
                else:
                    categoryscore = 2
                    
                df_emotion_frame.loc[index] = [fname, user_id, np.max(scores), np.argmax(scores), categoryscore]
                index+=1
                emotion_df.iloc[[0],[np.argmax(scores)]] += 1
            except:
                # print(fpath)
                index_failed +=1
            break
    print("count failed images for B_A2:", index_failed)
    print("count analysis images for B_A2", index)

In [154]:
task = 'part2_withuserID'
condition='B_A2'

In [155]:
folderpath = '/media/nali/Seagate Portable Drive/5_HRIStudy2_FrameData/face/' + task + '/condition_' + condition + '/'
folderpath

'/media/nali/Seagate Portable Drive/5_HRIStudy2_FrameData/face/part2_withuserID/condition_B_A2/'

In [157]:
allfiles = os.listdir(folderpath)
files = [ fname for fname in allfiles if fname.endswith('.jpg')]
print(len(files))
print(files[0:10])

5194
['10_frame_face_B_A2_0.jpg', '10_frame_face_B_A2_1.jpg', '10_frame_face_B_A2_10.jpg', '10_frame_face_B_A2_100.jpg', '10_frame_face_B_A2_101.jpg', '10_frame_face_B_A2_102.jpg', '10_frame_face_B_A2_103.jpg', '10_frame_face_B_A2_104.jpg', '10_frame_face_B_A2_105.jpg', '10_frame_face_B_A2_106.jpg']


In [158]:
import random
random.shuffle(files)

In [159]:
df_emotion_frame = pd.DataFrame(columns=['image_ID', 'user_id','score', 'emotion', 'category'])
df_emotion_frame

,image_ID,user_id,score,emotion,category


In [160]:
emotion_df = pd.DataFrame(emotion_data)
emotion_df

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


## 3. Facial emotion prediction

In [161]:
import datetime

In [162]:
print(datetime.datetime.now())
predictionEm(files,folderpath)
print(datetime.datetime.now())

2023-10-30 21:49:05.743818
amount images: 0
faild images: 0
amount images: 1000
faild images: 0
amount images: 2000
faild images: 0
amount images: 3000
faild images: 0
amount images: 4000
faild images: 0
amount images: 5000
faild images: 0
count failed images for B_A2: 0
count analysis images for B_A2 5115
2023-10-30 22:13:01.975360


In [163]:
df_emotion_frame.head()

,image_ID,user_id,score,emotion,category
0,70_frame_face_B_A2_1658.jpg,70,0.332313,3,1
1,7_frame_face_B_A2_5174.jpg,7,0.504808,4,2
2,65_frame_face_B_A2_4024.jpg,65,0.535947,5,0
3,17_frame_face_B_A2_913.jpg,17,0.499561,4,2
4,15_frame_face_B_A2_605.jpg,15,0.340792,1,0


In [164]:
emotion_df

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,807,380,41,1381,1663,796,47


In [143]:
emotion_df_A2 = emotion_df
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,5065,2477,268,7245,7312,5137,341


In [228]:
emotion_df_B_A2 = emotion_df
emotion_df_B_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,812,384,50,1394,1719,878,48


In [182]:
emotion_df_A2 = emotion_df
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,5072,2480,304,7252,7418,5181,350


In [229]:
emotion_df_B_A2 = emotion_df
emotion_df_B_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,812,384,50,1394,1719,878,48


In [109]:
emotion_df_A1_beforehelp = emotion_df
emotion_df_A1_beforehelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,3062,1448,45,1706,4223,2695,75


In [82]:
emotion_df_A1_withhelp = emotion_df
emotion_df_A1_withhelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,2950,1824,64,2221,5174,2724,133


In [125]:
emotion_df_B_A1 = emotion_df
emotion_df_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1075,750,10,642,1953,1020,32


In [40]:
emotion_df_A

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,385,635,31,1112,1065,193,84


In [45]:
emotion_df_B.head()

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1404,564,24,1119,2264,1764,104


In [232]:
df_emotion_frame.head()

,image_ID,emotion,category
0,10_frame_face_B_A2_0.jpg,1,0
1,10_frame_face_B_A2_1.jpg,3,1
2,10_frame_face_B_A2_10.jpg,3,1
3,10_frame_face_B_A2_100.jpg,3,1
4,10_frame_face_B_A2_101.jpg,3,1


In [110]:
df_emotion_frame_A1_beforehelp = df_emotion_frame
df_emotion_frame_A1_beforehelp.head()

,image_ID,user_id,score,emotion,category
0,40_frame_face_A1_beforehelp_3235.jpg,40,0.465912,4,2
1,23_frame_face_A1_beforehelp_10654.jpg,23,0.532831,5,0
2,76_frame_face_A1_beforehelp_1867.jpg,76,0.310138,0,0
3,63_frame_face_A1_beforehelp_4605.jpg,63,0.673332,1,0
4,63_frame_face_A1_beforehelp_3539.jpg,63,0.383551,0,0


In [85]:
df_emotion_frame_A1_withhelp = df_emotion_frame
df_emotion_frame_A1_withhelp.head()

,image_ID,score,emotion,category
0,30_frame_face_A1_withhelp_11673.jpg,0.476289,4,2
1,40_frame_face_A1_withhelp_8621.jpg,0.491490,4,2
2,62_frame_face_A1_withhelp_5501.jpg,0.402901,0,0
3,01_frame_face_A1_withhelp_0.jpg,0.661492,4,2
4,01_frame_face_A1_withhelp_1.jpg,0.639189,4,2


In [126]:
df_emotion_frame_B_A1 = df_emotion_frame
df_emotion_frame_B_A1.head()

,image_ID,user_id,score,emotion,category
0,11_frame_face_B_A1_0.jpg,11,0.374812,1,0
1,11_frame_face_B_A1_1.jpg,11,0.535233,4,2
2,11_frame_face_B_A1_10.jpg,11,0.547653,4,2
3,11_frame_face_B_A1_100.jpg,11,0.459249,1,0
4,11_frame_face_B_A1_101.jpg,11,0.355081,4,2


In [65]:
df_emotion_frame_A2

,image_ID,emotion,category


In [144]:
df_emotion_frame_A2 = df_emotion_frame
df_emotion_frame_A2

,image_ID,user_id,score,emotion,category
0,66_frame_face_A2_21116.jpg,66,0.545225,4,2
1,14_frame_face_A2_1056.jpg,14,0.538215,0,0
2,29_frame_face_A2_6901.jpg,29,0.320255,4,2
3,36_frame_face_A2_9587.jpg,36,0.504127,5,0
4,37_frame_face_A2_9871.jpg,37,0.567627,4,2
...,...,...,...,...,...
27840,2_frame_face_A2_7351.jpg,2,0.316987,1,0
27841,29_frame_face_A2_6160.jpg,29,0.450962,4,2
27842,21_frame_face_A2_3261.jpg,21,0.752215,3,1
27843,70_frame_face_A2_23115.jpg,70,0.440954,1,0


In [54]:
df_emotion_frame_A1_help = df_emotion_frame
df_emotion_frame_A1_help.head()

,image_ID,emotion,category
0,frame_face_A1_help_0.jpg,4,2
1,frame_face_A1_help_1.jpg,4,2
2,frame_face_A1_help_10.jpg,4,2
3,frame_face_A1_help_100.jpg,4,2
4,frame_face_A1_help_1000.jpg,2,0


In [43]:
df_emotion_frame_A['Condition'] ='A'
df_emotion_frame_A.head()

,image_ID,emotion,category,Condition
0,frame_A1138.jpg,4,2,A
1,frame_A989.jpg,3,1,A
2,frame_A2063.jpg,1,0,A
3,frame_A3233.jpg,4,2,A
4,frame_A1753.jpg,3,1,A


In [61]:
df_emotion_frame_B['Condition'] ='B'
df_emotion_frame_B.head()

,image_ID,emotion,category,Condition
0,frame_B491.jpg,3,1,B
1,frame_B78.jpg,3,1,B
2,frame_B1200.jpg,2,0,B
3,frame_B1009.jpg,3,1,B
4,frame_B918.jpg,6,1,B


In [127]:
emotion_df_B_A1['condition'] ='B_A1'
emotion_df_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,1075,750,10,642,1953,1020,32,B_A1


In [145]:
df_emotion_frame_A2['condition'] = 'A2'
df_emotion_frame_A2.head()

,image_ID,user_id,score,emotion,category,condition
0,66_frame_face_A2_21116.jpg,66,0.545225,4,2,A2
1,14_frame_face_A2_1056.jpg,14,0.538215,0,0,A2
2,29_frame_face_A2_6901.jpg,29,0.320255,4,2,A2
3,36_frame_face_A2_9587.jpg,36,0.504127,5,0,A2
4,37_frame_face_A2_9871.jpg,37,0.567627,4,2,A2


In [165]:
df_emotion_frame_B_A2 = df_emotion_frame
df_emotion_frame_B_A2['condition'] = 'B_A2'
df_emotion_frame_B_A2.head()

,image_ID,user_id,score,emotion,category,condition
0,70_frame_face_B_A2_1658.jpg,70,0.332313,3,1,B_A2
1,7_frame_face_B_A2_5174.jpg,7,0.504808,4,2,B_A2
2,65_frame_face_B_A2_4024.jpg,65,0.535947,5,0,B_A2
3,17_frame_face_B_A2_913.jpg,17,0.499561,4,2,B_A2
4,15_frame_face_B_A2_605.jpg,15,0.340792,1,0,B_A2


In [111]:
df_emotion_frame_A1_beforehelp['condition'] = 'A1_beforehelp'
df_emotion_frame_A1_beforehelp.head()

,image_ID,user_id,score,emotion,category,condition
0,40_frame_face_A1_beforehelp_3235.jpg,40,0.465912,4,2,A1_beforehelp
1,23_frame_face_A1_beforehelp_10654.jpg,23,0.532831,5,0,A1_beforehelp
2,76_frame_face_A1_beforehelp_1867.jpg,76,0.310138,0,0,A1_beforehelp
3,63_frame_face_A1_beforehelp_4605.jpg,63,0.673332,1,0,A1_beforehelp
4,63_frame_face_A1_beforehelp_3539.jpg,63,0.383551,0,0,A1_beforehelp


In [86]:
df_emotion_frame_A1_withhelp['condition'] = 'A1_withhelp'
df_emotion_frame_A1_withhelp.head()

,image_ID,score,emotion,category,condition
0,30_frame_face_A1_withhelp_11673.jpg,0.476289,4,2,A1_withhelp
1,40_frame_face_A1_withhelp_8621.jpg,0.491490,4,2,A1_withhelp
2,62_frame_face_A1_withhelp_5501.jpg,0.402901,0,0,A1_withhelp
3,01_frame_face_A1_withhelp_0.jpg,0.661492,4,2,A1_withhelp
4,01_frame_face_A1_withhelp_1.jpg,0.639189,4,2,A1_withhelp


In [128]:
df_emotion_frame_B_A1['condition'] = 'B_A1'
df_emotion_frame_B_A1.head()

,image_ID,user_id,score,emotion,category,condition
0,11_frame_face_B_A1_0.jpg,11,0.374812,1,0,B_A1
1,11_frame_face_B_A1_1.jpg,11,0.535233,4,2,B_A1
2,11_frame_face_B_A1_10.jpg,11,0.547653,4,2,B_A1
3,11_frame_face_B_A1_100.jpg,11,0.459249,1,0,B_A1
4,11_frame_face_B_A1_101.jpg,11,0.355081,4,2,B_A1


In [234]:
df_emotion_frame_B_A2['condition'] = 'B_A2'
df_emotion_frame_B_A2.head()

,image_ID,emotion,category,condition
0,10_frame_face_B_A2_0.jpg,1,0,B_A2
1,10_frame_face_B_A2_1.jpg,3,1,B_A2
2,10_frame_face_B_A2_10.jpg,3,1,B_A2
3,10_frame_face_B_A2_100.jpg,3,1,B_A2
4,10_frame_face_B_A2_101.jpg,3,1,B_A2


In [52]:
emotion_df_A1 = emotion_df

In [100]:
df_emotion_frame_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1033,622,38,1598,1523,892,79


In [112]:
emotion_df_A2 = emotion_df
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1346,530,14375,1648,1638,1179,978


In [53]:
emotion_df_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1427,617,15711,791,1815,1778,2044


In [112]:
emotion_df_A1_beforehelp['condition'] ='A1_beforehelp'
emotion_df_A1_beforehelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,3062,1448,45,1706,4223,2695,75,A1_beforehelp


In [87]:
emotion_df_A1_withhelp['condition'] ='A1_withhelp'
emotion_df_A1_withhelp

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,2950,1824,64,2221,5174,2724,133,A1_withhelp


In [129]:
emotion_df_B_A1['condition'] ='B_A1'
emotion_df_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,1075,750,10,642,1953,1020,32,B_A1


In [166]:
emotion_df_B_A2['condition'] ='B_A2'
emotion_df_B_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,0,0,0,0,0,0,0,B_A2


In [146]:
emotion_df_A2['condition'] ='A2'
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,5065,2477,268,7245,7312,5137,341,A2


In [136]:
df_emotion_frame_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A2_facialEmotion_score_new.csv", index=False)

In [137]:
emotion_df_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A2_facialEmotion_sum_new.csv", index=False)

In [118]:
df_emotion_frame_B_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2_B_A2_facialEmotion_score_new.csv", index=False)

In [114]:
df_emotion_frame_A1_beforehelp.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A1_beforehelp_facialEmotion_score_userid_2.csv", index=False)

In [113]:
emotion_df_A1_beforehelp.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A1_beforehelp_facialEmotion_sum_userid_2.csv", index=False)

In [88]:
df_emotion_frame_A1_withhelp.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A1_withhelp_facialEmotion_score_userid_2.csv", index=False)

In [89]:
emotion_df_A1_withhelp.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A1_withhelp_facialEmotion_sum_userid_2.csv", index=False)

In [130]:
df_emotion_frame_B_A1.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_B_A1_facialEmotion_score_userid_2.csv", index=False)

In [131]:
emotion_df_B_A1.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A1_withhelp_facialEmotion_sum_userid_2.csv", index=False)

In [147]:
df_emotion_frame_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A2_facialEmotion_score_userid_2.csv", index=False)

In [148]:
emotion_df_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/part2_A2_facialEmotion_score_sum_userid_2.csv", index=False)

In [236]:
df_emotion_frame_B_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A2_facialEmotion_score_userid.csv", index=False)

In [237]:
emotion_df_B_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A2_facialEmotion_score_sum_userid.csv", index=False)

In [63]:
df_emotion_frame_B.to_csv("/home/nali/Develop/HRI/data/HRI_Study1/B_facialEmotionCategories_2285.csv", index=False)

In [44]:
df_emotion_frame_B.to_csv("/home/nali/Develop/HRI/data/HRI_Study2/facial_Emotion/Part2_B_facialEmotionCategories_11218.csv", index=False)

In [54]:
df_emotion_frame_A1.to_csv("/home/nali/Develop/HRI/data/HRI_Study2/facial_Emotion/Part2_A1_facialEmotionCategories.csv", index=False)

In [64]:
emotion_df_B.to_csv("/home/nali/Develop/HRI/data/HRI_Study1/B_facialEmotion_SUM_2285.csv", index=False)

In [119]:
df_emotion_frame_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2/Part2_A2_facialEmotionCategories.csv", index=False)

In [117]:
emotion_df_A2['Condition'] =['A2']
emotion_df_A2

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Condition
0,1346,530,14375,1648,1638,1179,978,A2


In [118]:
emotion_df_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2/part2_A2_facialEmotion.csv", index=False)

In [215]:
emotion_df_B

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Condition
0,393,77,3688,228,429,519,354,B


In [126]:
emotion_df_B['Condition'] =['B']
emotion_df_B

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Condition
0,1033,622,38,1598,1523,892,79,B


In [217]:
emotion_df_B.to_csv("/home/nali/Develop/HRI/data/HRI_Study2/part3_B_facialEmotion.csv", index=False)

In [141]:
#combine two categories csv file
# emotion_df = emotion_df_A
emotion_df = pd.concat([emotion_df_A, emotion_df_B], axis=0)

emotion_df
# df.append(df2, ignore_index=True)

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Condition
0,0,0,0,0,0,0,0,NaN
0,284,126,3078,344,317,250,235,B


## 2nd Algorithm for emotion detection on frame data --- FER 2023
https://pypi.org/project/fer/

In [7]:
from fer import FER
import cv2
import datetime

In [8]:
emotion_fourCategories = {0: 'Negative', 1: 'Positive', 2: 'Neutral', 3: 'Surprise'}

In [9]:
df_emotion_frame = pd.DataFrame(columns=['image_ID', 'user_id', 'emotion_scores', 'emotion_index', 'emotion_category'])
df_emotion_frame

,image_ID,user_id,emotion_scores,emotion_index,emotion_category


In [10]:
df_emotion_frame = pd.DataFrame({'image_ID':[0], 'user_id':[0], 'emotion_scores':[0], 'emotion_index':[0], 'emotion_category':[0]})
df_emotion_frame

,image_ID,user_id,emotion_scores,emotion_index,emotion_category
0,0,0,0,0,0


In [18]:
emotion_df = pd.DataFrame(emotion_data)
emotion_df

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [90]:
emotion, score = detector.top_emotion(img)
emotion
print(score)
np.argmax(0.91)
emotion_index[emotion]

0.34


5

In [19]:
df_emotion_frame.head()

,image_ID,user_id,emotion_scores,emotion_index,emotion_category
0,0,0,0,0,0


In [20]:
emotion_df.head()

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,0,0,0


In [21]:
emotion_index = {'anger':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4,'sad':5, 'surprise':6}

In [22]:
def predictionEm_fer(files,folderpath):
    
    index = 0
    index_failed = 0
    
    detector = FER(mtcnn=True)
    for fname in files:
        
        if index%1000 == 0:
            print("amount images:", index)
            print("faild images:", index_failed)
        
        fpath = folderpath + fname
        # print(fpath)
        user_id = fname.split('_')[0]
            
        try:
            emotion, scores = detector.top_emotion(fpath)
            if emotion_index[emotion] in (0, 1, 2, 5):
                categoryscore = 0
            elif emotion_index[emotion] == 3:
                categoryscore = 1
            elif emotion_index[emotion] == 6:
                categoryscore = 3
            else:
                categoryscore = 2
            
            df_emotion_frame.loc[index] = [fname, user_id, scores, emotion_index[emotion], categoryscore]
            index+=1
            emotion_df.iloc[[0],[emotion_index[emotion]]] += 1
        except:
            # print(fpath)
            index_failed +=1
            
    print("count failed images:", index_failed)
    print("count emotion images:", index)

In [23]:
folderpath = '/media/nali/Seagate Portable Drive/5_HRIStudy2_FrameData/face/part2_withuserID/condition_B_A1/'

In [24]:
allfiles = os.listdir(folderpath)
files = [ fname for fname in allfiles if fname.endswith('.jpg')]
print(len(files))
print(files[:10])

5572
['11_frame_face_B_A1_0.jpg', '11_frame_face_B_A1_1.jpg', '11_frame_face_B_A1_10.jpg', '11_frame_face_B_A1_100.jpg', '11_frame_face_B_A1_101.jpg', '11_frame_face_B_A1_102.jpg', '11_frame_face_B_A1_103.jpg', '11_frame_face_B_A1_104.jpg', '11_frame_face_B_A1_105.jpg', '11_frame_face_B_A1_106.jpg']


In [25]:
print(datetime.datetime.now())
predictionEm_fer(files,folderpath)
print(datetime.datetime.now())
print("finally....done!!!")

2023-10-27 10:04:30.600023


2023-10-27 10:04:31.862336: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-27 10:04:31.862357: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nali-Alienware-m15-R2): /proc/driver/nvidia/version does not exist
2023-10-27 10:04:31.862992: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


amount images: 0
faild images: 0
amount images: 0
faild images: 1
amount images: 0
faild images: 2
amount images: 0
faild images: 3
amount images: 0
faild images: 4
amount images: 0
faild images: 5
amount images: 0
faild images: 6
amount images: 0
faild images: 7
amount images: 0
faild images: 8
amount images: 0
faild images: 9
amount images: 0
faild images: 10
amount images: 0
faild images: 11
amount images: 1000
faild images: 627
amount images: 2000
faild images: 897
amount images: 3000
faild images: 1412
count failed images: 1857
count emotion images: 3715
2023-10-27 11:24:51.888652
finally....done!!!


In [26]:
df_emotion_frame.head()

,image_ID,user_id,emotion_scores,emotion_index,emotion_category
0,11_frame_face_B_A1_108.jpg,11,0.69,4,2
1,11_frame_face_B_A1_109.jpg,11,0.75,4,2
2,11_frame_face_B_A1_138.jpg,11,0.34,3,1
3,12_frame_face_B_A1_176.jpg,12,0.75,3,1
4,12_frame_face_B_A1_191.jpg,12,0.46,4,2


In [34]:
df_frame_B_A1 = df_emotion_frame

In [35]:
df_frame_B_A1.head()

,image_ID,user_id,emotion_scores,emotion_index,emotion_category,condition
0,11_frame_face_B_A1_108.jpg,11,0.69,4,2,B_A1
1,11_frame_face_B_A1_109.jpg,11,0.75,4,2,B_A1
2,11_frame_face_B_A1_138.jpg,11,0.34,3,1,B_A1
3,12_frame_face_B_A1_176.jpg,12,0.75,3,1,B_A1
4,12_frame_face_B_A1_191.jpg,12,0.46,4,2,B_A1


In [36]:
df_frame_B_A1['condition'] = 'B_A1'

In [37]:
df_sum_B_A1 = emotion_df

In [38]:
df_sum_B_A1['condition'] = 'B_A1'

In [39]:
df_sum_B_A1

,Anger,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,condition
0,0,13,115,902,1435,1221,29,B_A1


In [41]:
df_frame_B_A1.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A1_FER_facialEmotion_score_userid.csv", index=False)

In [40]:
df_sum_B_A1.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A1_FER_facialEmotion_sum_userid.csv", index=False)

In [31]:
df_frame_B_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A2_FER_facialEmotion_score_userid.csv", index=False)

In [32]:
df_sum_B_A2.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/facial_Emotion/part2/withuserid/part2_B_A2_FER_facialEmotion_sum_userid.csv", index=False)

## 